In [78]:
import pandas as pd


activitats = pd.read_csv('activitats.csv', encoding = 'latin-1')
notes = pd.read_csv('notes.csv', sep=';', encoding = 'latin-1')
notes.rename(columns = {notes.columns[0]: 'userid'}, inplace = True)
trameses = pd.read_csv('trameses.csv', sep=',', encoding = 'latin-1')
trameses.rename(columns = {trameses.columns[0]: 'tramesa_id'}, inplace = True)

In [79]:
activitats

,activitat_id,activitat,aula_id,startdate,duedate,grade
0,3,Problema 1.1 - Hello world!!!,87,0,0,100
1,4,Problema 1.2 - Hello world!!! ++,87,0,0,100
2,5,Problema 3a.1: Càlcul edat,87,0,0,100
3,6,Problema 3a.8: Conversió dòlars a euros,87,0,0,100
4,7,Problema 3a.11: Mitjana de notes,87,0,0,100
...,...,...,...,...,...,...
1796,4194,(OPCIONAL) Problema 9.24 Triple pitagòric,184,0,0,100
1797,4379,VPLParent,114,0,0,10
1798,4380,VPL Child - Duplicate Me as Needed,114,0,0,10
1799,4386,Lliurament Pràctica 5: NEW La Major,176,1571985000,1576745700,100


In [80]:
notes

,userid,aula_id,P_Grade,P_Grade_Date,F_Grade,F_Grade_Date,R_Grade,R_Grade_Date
0,176,92,NaN,NaN,"9,25",1667829600,NaN,1675173600
1,179,92,NaN,NaN,"7,75",1667829600,NaN,1675173600
2,168,92,NaN,NaN,"7,75",1667829600,NaN,1675173600
3,401,92,NaN,NaN,"3,50",1667829600,"8,25",1675173600
4,482,92,NaN,NaN,"7,00",1667829600,NaN,1675173600
...,...,...,...,...,...,...,...,...
1083,2818,184,NaN,1.730988e+09,NaN,1736949600,NaN,1738072800
1084,2822,184,NaN,1.730988e+09,NaN,1736949600,NaN,1738072800
1085,2836,184,NaN,1.730988e+09,NaN,1736949600,NaN,1738072800
1086,2982,184,NaN,1.730988e+09,NaN,1736949600,NaN,1738072800


In [81]:
trameses

,tramesa_id,activitat_id,userid,datesubmitted,grader,dategraded,grade,nevaluations
0,6,3,31,1658829978,0.0,1.658830e+09,100.0,1
1,7,362,26,1658837049,0.0,1.658837e+09,100.0,0
2,9,362,26,1658837075,0.0,1.658837e+09,100.0,0
3,15,362,31,1662537577,0.0,1.662538e+09,NaN,0
4,16,362,31,1662537593,0.0,1.662712e+09,100.0,0
...,...,...,...,...,...,...,...,...
547957,1235631,3567,2644,1729605745,0.0,1.729606e+09,60.0,17
547958,1235635,3573,1828,1729605824,0.0,0.000000e+00,NaN,0
547959,1235643,3573,1828,1729605880,0.0,0.000000e+00,NaN,1
547960,1235646,3567,2650,1729605935,0.0,1.729606e+09,100.0,6


In [82]:
Act_Not = pd.merge(notes, activitats, on='aula_id', how='outer')
activitats_valides = Act_Not[Act_Not['F_Grade_Date'] > Act_Not['duedate']]['activitat_id'].unique()

#CLEAN TRAMESAS
#actividades validas en tramesas
trameses_1 = trameses[trameses['activitat_id'].isin(activitats_valides)]
trameses_1 = trameses_1.dropna(subset='grade')
trameses_filtered = trameses_1.groupby(['userid', 'activitat_id'])[['grade', 'nevaluations']].max()
#eliminar naans trameses
trameses_filtered

grade  nevaluations
userid activitat_id                     
21     362           100.0             1
       363           100.0             6
       366           100.0             3
       367           100.0             5
       368           100.0             4
...                    ...           ...
3008   3530          100.0             7
       3531          100.0             2
       3532          100.0             5
       3565           80.0            12
       3568           10.0            16

[97858 rows x 2 columns]

In [83]:
merged_df = pd.merge(trameses_filtered.reset_index(), activitats, on='activitat_id')
merged_df.rename(columns={'grade_x': 'grade', 'grade_y': 'gradeMax'}, inplace=True)
merged_df.drop(['activitat', 'startdate', 'duedate'], axis=1, inplace=True)
merged_df['grade'] = merged_df['grade'] / merged_df['gradeMax']
merged_df = merged_df.drop('gradeMax', axis=1)
merged_df

,userid,activitat_id,grade,nevaluations,aula_id
0,21,362,1.0,1,92
1,21,363,1.0,6,92
2,21,366,1.0,3,92
3,21,367,1.0,5,92
4,21,368,1.0,4,92
...,...,...,...,...,...
97853,3008,3530,1.0,7,176
97854,3008,3531,1.0,2,176
97855,3008,3532,1.0,5,176
97856,3008,3565,0.8,12,176


In [84]:
merged_df = merged_df.groupby(['userid', 'aula_id']).agg(
    grade_mean=('grade', 'mean'),
    Totalnevaluations=('nevaluations', 'sum'),
    Totalactividad_count=('activitat_id', 'size')
).reset_index()

merged_df

,userid,aula_id,grade_mean,Totalnevaluations,Totalactividad_count
0,21,92,0.862911,266,79
1,26,87,0.909091,9,11
2,26,92,1.000000,15,7
3,26,141,0.931034,14,58
4,26,143,0.986667,44,9
...,...,...,...,...,...
1100,2944,176,0.996610,220,59
1101,2982,184,0.984375,265,80
1102,2984,184,0.982143,129,70
1103,2999,176,0.903654,128,52


In [85]:
#save to csv
merged_df.to_csv('Clean.csv', index=False)